# xmon1 <-> xmon2 <-> (xmon3, xmon4)

In [46]:
import numpy as np
import numpy.linalg as alg
import scqubits as scq
import sys
sys.path.insert(0, '..')
from QSwitch import QSwitch

In [47]:
hbar = 1.054e-34
h = hbar*2*np.pi
qe = 1.609e-19
flux_quant = h/2/qe
red_flux_quant = hbar/2/qe 

# Unit conversions
MHz = 10.0**(-3)
GHz = 1.0
kHz = 10.0**(-6)
us = 10.0**3
ns = 1.0

In [110]:
# Q3D VALUES
# All in SI units
Cjs = np.array([76.694, 127.669, 66.517, 86.189])*1e-15 # fF -> F
Cab = np.array([4.185, 4.142, 4.345])*1e-15 # [C12, C23, C24] fF -> F
Crqs = np.array([10.054, 19.666, 9.601, 10.807])*1e-15 # fF -> F
Cds = np.array([72, 135, 72, 62])*1e-18 # aF -> F

# HFSS VALUES
res_freqs = np.array([7.073, 5.545, 6.178, 6.410]) # GHz
Q_res = np.array([1.272e5, 2.860e5, 7.677e4, 1.526e5])
Q_drives = np.array([np.inf, 8.159e6, np.inf, np.inf]) # currently only simulated for qubit 2

# GET ZO FROM CPW MATHEMATICA
Zo = 52.7066 # Ohm

Ljs = np.array([10.894, 12.570, 16.341, 11.672])*1e-9 # nH -> H

c_matrix = np.array([
    [Cab[0]+Cjs[0], -Cab[0], 0, 0],
    [-Cab[0], sum(Cab)+Cjs[1], -Cab[1], -Cab[2]],
    [0, -Cab[1], Cab[1]+Cjs[2], 0],
    [0, -Cab[2], 0, Cab[2]+Cjs[3]],
])

inv_l_matrix = np.diag(1/Ljs)

print('Check symmetric:', np.allclose(c_matrix, c_matrix.T), np.allclose(inv_l_matrix, inv_l_matrix.T))

Check symmetric: True True


### simultaneous diagonalization

In [111]:
inv_c_matrix = alg.inv(c_matrix)

evals_c, u1 = alg.eigh(inv_c_matrix)
u1 = np.array([u1[i]/np.sqrt(evals_c[i]) for i in range(len(evals_c))])
evals, evecs = alg.eigh(u1.transpose() @ inv_l_matrix @ u1)
print('Modes (cols are evecs)\n', np.around(evecs, 3))

# ======== IDK WHY BUT THIS DOESN'T WORK ========
# Look for col with max in Q1_NODE -> use q_modes to reference in EC, EJ
# q_modes = np.array([np.argmax(np.abs(evecs[i,:])) for i in range(4)])
# print('Qubit modes were mapped', f'q1: mode {q_modes[0]},', f'q2: mode {q_modes[1]},', f'q3: mode {q_modes[2]},', f'q4: mode {q_modes[3]}')
# old_evecs = evecs.copy()
# for i in range(4):
#     evecs[:,i] = old_evecs[:,q_modes[i]]
# # evals = np.array([evals[q_modes[i]] for i in range(4)])
# ECs = qe**2/2/h * evecs.transpose() @ inv_c_matrix @ evecs
# EJs = flux_quant**2/h/4/np.pi**2 * evecs.transpose() @ inv_l_matrix @ evecs
# ===============================================

ECs = qe**2/2/h * inv_c_matrix
EJs = flux_quant**2/h/4/np.pi**2 * inv_l_matrix
Zs = np.sqrt(8*np.diag(ECs)/np.diag(EJs))
gs = np.array([[4*ECs[i,j]/np.sqrt(Zs[i]*Zs[j]) for i in range(len(c_matrix))] for j in range(len(c_matrix))])

print('all ECs\n', 1e-9*ECs)
print('all EJs\n', 1e-9*ECs)
print('all q-q gs\n', 1e-6*gs)
print('impedances (unitless)\n', *Zs)

ECs = 1e-9 * np.diag(ECs)
EJs = 1e-9 * np.diag(EJs)
gs = 1e-9 * np.abs(np.array([gs[0,1], gs[1,2], gs[1,3]])) # g12, g23, g24

# AFTER HERE EVERYTHING IS INDEXED SO QUBIT NUMBER = INDEX AND EVERYTHING IS IN GHz


Modes (cols are evecs)
 [[-0.058  0.085  0.992 -0.069]
 [ 0.017  0.996 -0.082  0.037]
 [ 0.029  0.031 -0.07  -0.997]
 [ 0.998 -0.013  0.061  0.025]]
all ECs
 [[0.24204641 0.00724117 0.00042447 0.00034753]
 [0.00724117 0.13994223 0.00820335 0.00671625]
 [0.00042447 0.00820335 0.2771076  0.0003937 ]
 [0.00034753 0.00671625 0.0003937  0.21622097]]
all EJs
 [[0.24204641 0.00724117 0.00042447 0.00034753]
 [0.00724117 0.13994223 0.00820335 0.00671625]
 [0.00042447 0.00820335 0.2771076  0.0003937 ]
 [0.00034753 0.00671625 0.0003937  0.21622097]]
all q-q gs
 [[2682.96475335   88.81274226    4.11014494    3.89460657]
 [  88.81274226 1899.17979178   87.89171471   83.28262269]
 [   4.11014494   87.89171471 2343.92780049    3.85421778]
 [   3.89460657   83.28262269    3.85421778 2449.82801981]]
impedances (unitless)
 0.3608640931885057 0.2947424511402884 0.4728944235090966 0.3530386137389365


In [113]:
cutoffs = [4, 5, 4, 4]
isCavity = [False, False, False, False]

qram = QSwitch(
    EJs=EJs,
    ECs=ECs,
    gs=gs,
    cutoffs=cutoffs,
    isCavity=isCavity,
)

qubit_freqs = np.array(qram.qubit_freqs)
alphas = qram.alphas
charge_mat_el = 1/np.sqrt(2) * (EJs/8/ECs)**1/4
g_01s = qe * Crqs/(Crqs+Cjs) * res_freqs * np.sqrt(2*Zo/hbar) * charge_mat_el
g_rqs = Crqs/(Crqs+Cjs) * res_freqs * np.sqrt(2*Zo/(h/qe**2))
deltas = qubit_freqs-res_freqs
lamb_shifts = g_rqs**2/deltas
chi_shifts = g_rqs**2/deltas * alphas/(deltas+alphas)
drive_purcell_rate_analytical = (2*np.pi*qubit_freqs*1e9*Cds)**2/(Cjs+Cds) * 50 /2/np.pi
drive_purcell_rate_sim = qubit_freqs*1e9/Q_drives
kappas = res_freqs/Q_res
res_purcell_rate = kappas * (g_01s/deltas)**2

print('EC values (GHz):\n\t', *ECs)
print('EJ values (GHz):\n\t', *EJs)
print('qubit freqs/2pi (GHz):\n\t', *qubit_freqs)
print('alphas/2pi (GHz):\n\t', *alphas)
print('q-q couplings g12, g23, g24 (MHz)\n\t', *(gs*1e3))
print('q-res couplings (MHz) [want 50-100]:\n\t', *(1e3*g_rqs))
print('delta wq-wr (GHz) [want 1.5-2]:\n\t', *deltas)
print('lamb shifts (MHz):\n\t', *(1e3*lamb_shifts))
print('chi shifts (MHz) [want 0.15-0.5]:\n\t', *(1e3*chi_shifts))
print('\n')
print('drive purcell limits - analytical (ms) [want >1 ms]:\n\t', *(1e3/drive_purcell_rate_analytical))
print('drive purcell limits - simulated (ms) [want >1 ms]:\n\t', *(1e3/drive_purcell_rate_sim))
print('res linewidth kappas - simulated (kHz):\n\t', *(1e6*kappas))
print('res purcell limits - simulated (s):\n\t', *(1/res_purcell_rate))

EC values (GHz):
	 0.24204641069391794 0.1399422267463943 0.27710759648940253 0.21622097200303578
EJ values (GHz):
	 14.869668686887596 12.887046195302583 9.913112457925065 13.878527302514861
qubit freqs/2pi (GHz):
	 5.111407419491645 3.652683164225361 4.391154341382721 4.672567698878251
alphas/2pi (GHz):
	 -0.2734410007153798 -0.15399422156579679 -0.33128181755694186 -0.2434725859171163
q-q couplings g12, g23, g24 (MHz)
	 88.81274225683256 87.89171471023947 83.2826226886706
q-res couplings (MHz) [want 50-100]:
	 52.62309306159774 47.51218037265018 50.02306614250445 45.84612764869781
delta wq-wr (GHz) [want 1.5-2]:
	 -1.9615925805083556 -1.892316835774639 -1.7868456586172785 -1.737432301121749
lamb shifts (MHz):
	 -1.4117049334739675 -1.192933044343577 -1.4004047491341503 -1.2097550039927614
chi shifts (MHz) [want 0.15-0.5]:
	 -0.17271239813435202 -0.08977364163909862 -0.21902771944891922 -0.1486907226467507


drive purcell limits - analytical (ms) [want >1 ms]:
	 1.8041516840766467 1.

 /Users/conniemiao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py: 36